In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("winson13/heart-disease-dataset")

print("Path to dataset files:", path)

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/morning/.cache/kagglehub/datasets/winson13/heart-disease-dataset/versions/2


In [2]:
import os
os.listdir(path)

['Dataset Heart Disease.csv', 'cleveland1.csv', 'cleveland2.csv']

In [3]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv(f'{path}/cleveland2.csv')
df.head()

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
df.shape

(302, 15)

In [6]:
df.drop('Unnamed: 0',axis=1, inplace=True)

In [7]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       302 non-null    int64  
 1   sex       302 non-null    int64  
 2   cp        302 non-null    int64  
 3   trestbps  302 non-null    int64  
 4   chol      302 non-null    int64  
 5   fbs       302 non-null    int64  
 6   restecg   302 non-null    int64  
 7   thalach   302 non-null    int64  
 8   exang     302 non-null    int64  
 9   oldpeak   302 non-null    float64
 10  slope     302 non-null    int64  
 11  ca        302 non-null    int64  
 12  thal      302 non-null    int64  
 13  target    302 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.2 KB


In [9]:
df['sex'].unique()

array([1, 0])

In [10]:
y = df['target'].values
X = df.drop('target', axis=1)

In [11]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(302, 13)
(302, 13)


In [12]:
# build mask
gender_0_mask = df['sex'] == 0
gender_1_mask = df['sex'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  96
Male:  206


In [13]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=seed)

In [14]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 4 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 5 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
   Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0     1  Female  0.916667  0.875000  0.125000  0.083333      11       7   
1     2  Female  1.000000  0.333333  0.666667  0.000000      13       2   
2     3  Female  0.933333  1.000000  0.000000  0.066667      14       4   
3     4  Female  1.000000  1.000000  0.000000  0.000000      16       3   
4     5  Female  1.000000  0.666667  0.333333  0.000000      16       2   
5     1    Male  0.666667  0.708333  0.291667  0.333333      12      17   
6     2    Male  0.789474  0.681818  0.318182  0.210526      15      15   
7     3    Male  0.727273  0.789474  0.210526  0.272727      16      15   
8     4    Male  0.684211  0.727273  0.272727  0.315789      13      16   
9     5    Male  0.928571  0.925926  0.074074  0.071429      13      25   

   SVM_FP  SVM_FN  ...  ANN_FP  ANN_FN    NB_TPR    NB_TNR    NB_FPR  \

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
results_df = pd.concat(results_list, ignore_index=True)

In [16]:
result_path = './results/k30_result.xlsx'
results_df.to_excel(result_path, index=False)

In [17]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.916667,0.875000,0.125000,0.083333,11,7,1,1,...,0,1,0.833333,1.000000,0.000000,0.166667,10,8,0,2
1,2,Female,1.000000,0.333333,0.666667,0.000000,13,2,4,0,...,4,0,1.000000,0.500000,0.500000,0.000000,13,3,3,0
2,3,Female,0.933333,1.000000,0.000000,0.066667,14,4,0,1,...,0,1,0.933333,1.000000,0.000000,0.066667,14,4,0,1
3,4,Female,1.000000,1.000000,0.000000,0.000000,16,3,0,0,...,0,0,1.000000,1.000000,0.000000,0.000000,16,3,0,0
4,5,Female,1.000000,0.666667,0.333333,0.000000,16,2,1,0,...,1,0,1.000000,0.666667,0.333333,0.000000,16,2,1,0


In [18]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(24.0), pvalue=np.float64(0.02000818529160622))
SVM - FPR: TtestResult(statistic=np.float64(-0.06321645242138421), pvalue=np.float64(0.951145124181315), df=np.float64(8.0))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(8.0), pvalue=np.float64(0.39761475195653073))
DT -TPR: TtestResult(statistic=np.float64(3.472305150203641), pvalue=np.float64(0.008414192792365513), df=np.float64(8.0))
DT - FPR: TtestResult(statistic=np.float64(2.7371942826877036), pvalue=np.float64(0.02555986286682494), df=np.float64(8.0))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(6.0), pvalue=np.float64(0.2222222222222222))
RF -TPR: MannwhitneyuResult(statistic=np.float64(25.0), pvalue=np.float64(0.009700785068229596))
RF - FPR: TtestResult(statistic=np.float64(1.0497258197530002), pvalue=np.float64(0.32451806672776934), df=np.float64(8.0))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(2.0), pvalue=np.float64(0.031141210595796758))
LR -TPR: Mannwh